# Installing Library

In [ ]:
!pip install torch
!pip install hiddenlayer

# Importing Library

In [ ]:
import torch,os,sys,torchvision,argparse
import torchvision.transforms as tfs
import time,math
import numpy as np
from torch.backends import cudnn
from torch import optim
import torch,warnings
from torch import nn
import torchvision.utils as vutils

import torch.nn.functional as F

import torch.utils.data as data
import torchvision.transforms as tfs
from torchvision.transforms import functional as FF
import os,sys
sys.path.append('.')
sys.path.append('..')
import numpy as np
import torch
import random
from PIL import Image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torchvision.utils import make_grid

from torchvision.models import vgg16

from math import exp
from torch.autograd import Variable

from math import exp
from  torchvision.transforms import ToPILImage


In [ ]:
parser=argparse.ArgumentParser()
parser.add_argument('--steps',type=int,default=100000)
parser.add_argument('--device',type=str,default='Automatic detection')
parser.add_argument('--resume',type=bool,default=True)
parser.add_argument('--eval_step',type=int,default=1000)
parser.add_argument('--lr', default=0.0001, type=float, help='learning rate')
parser.add_argument('--model_dir',type=str,default='./trained_models_best/')
parser.add_argument('--trainset',type=str,default='ots_train')
parser.add_argument('--testset',type=str,default='ots_test')
parser.add_argument('--net',type=str,default='mod_pdr_net')
parser.add_argument('--gps',type=int,default=3,help='residual_groups')
parser.add_argument('--blocks',type=int,default=16,help='residual_blocks')
parser.add_argument('--bs',type=int,default=1,help='batch size')
parser.add_argument('--crop',action='store_true')
parser.add_argument('--crop_size',type=int,default=240,help='Takes effect when using --crop ')
parser.add_argument('--no_lr_sche',action='store_true',help='no lr cos schedule')
parser.add_argument('--perloss',action='store_true',help='perceptual loss')

opt=parser.parse_args('')
opt.device='cuda' if torch.cuda.is_available() else 'cpu'
model_name=opt.trainset+'_'+opt.net.split('.')[0]
opt.model_dir=opt.model_dir+model_name+'.pk'
last_save_model_dir = './trained_models_last_saved/'+model_name+'.pk'
print('last save model": ',  last_save_model_dir)
log_dir='logs/'+model_name

print(opt)
print('model_dir:',opt.model_dir)


if not os.path.exists('trained_models_best'):
    os.mkdir('trained_models_best')
if not os.path.exists('trained_models_last_saved'):
    os.mkdir('trained_models_last_saved')
if not os.path.exists('numpy_files'):
    os.mkdir('numpy_files')
if not os.path.exists('logs'):
    os.mkdir('logs')
if not os.path.exists('samples'):
    os.mkdir('samples')
if not os.path.exists(f"samples/{model_name}"):
    os.mkdir(f'samples/{model_name}')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

last save model":  ./trained_models_last_saved/ots_train_custom_model_modif_data_424_3_16.pk
Namespace(blocks=16, bs=1, crop=False, crop_size=240, device='cuda', eval_step=1000, gps=3, lr=0.0001, model_dir='./trained_models_best/ots_train_custom_model_modif_data_424_3_16.pk', net='custom_model_modif_data_424', no_lr_sche=False, perloss=False, resume=True, steps=100000, testset='ots_test', trainset='ots_train')
model_dir: ./trained_models_best/ots_train_custom_model_modif_data_424_3_16.pk


# Data Utils

In [ ]:
BS=opt.bs
print(BS)
crop_size='whole_img'
if opt.crop:
    crop_size=opt.crop_size

def tensorShow(tensors,titles=None):
        '''
        t:BCWH
        '''
        fig=plt.figure()
        for tensor,tit,i in zip(tensors,titles,range(len(tensors))):
            img = make_grid(tensor)
            npimg = img.numpy()
            ax = fig.add_subplot(211+i)
            ax.imshow(np.transpose(npimg, (1, 2, 0)))
            ax.set_title(tit)
        plt.show()

class RESIDE_Dataset(data.Dataset):
    def __init__(self,path,train,size=crop_size,format='.png'):
        super(RESIDE_Dataset,self).__init__()
        self.size=size
        print('crop size',size)
        self.train=train
        self.format=format
        self.haze_imgs_dir=os.listdir(os.path.join(path,'hazy'))
        self.haze_imgs=[os.path.join(path,'hazy',img) for img in self.haze_imgs_dir]
        self.clear_dir=os.path.join(path,'GT')
        print(self.clear_dir)

    def __getitem__(self, index):
        haze=Image.open(self.haze_imgs[index])
        if isinstance(self.size,int):
            while haze.size[0]<self.size or haze.size[1]<self.size :
                index=random.randint(0,20000)
                haze=Image.open(self.haze_imgs[index])
        img=self.haze_imgs[index]
        id = img.split('/')[-1].split('_')
        
        if len(id) == 3 :
          clear_name = id[0]+"_"+id[1]+"_GT"+self.format
        if len(id) == 4 :
          clear_name = id[0]+"_"+id[1]+"_"+id[2]+"_GT"+self.format
        if len(id) == 2 :
          clear_name = id[0]+"_GT."+id[-1].split('.')[-1]
        clear=Image.open(os.path.join(self.clear_dir,clear_name))
        clear=tfs.CenterCrop(haze.size[::-1])(clear)
        if not isinstance(self.size,str):
            i,j,h,w=tfs.RandomCrop.get_params(haze,output_size=(self.size,self.size))
            haze=FF.crop(haze,i,j,h,w)
            clear=FF.crop(clear,i,j,h,w)

        
        haze,clear=self.augData(haze.convert("RGB") ,clear.convert("RGB") )
        return haze,clear
    def augData(self,data,target):
        data =tfs.Resize((240,240))(data)
        target =tfs.Resize((240,240))(target)
        if self.train:
            rand_hor=random.randint(0,1)
            rand_rot=random.randint(0,3)
            data=tfs.RandomHorizontalFlip(rand_hor)(data)
            target=tfs.RandomHorizontalFlip(rand_hor)(target)
            if rand_rot:
                data=FF.rotate(data,90*rand_rot)
                target=FF.rotate(target,90*rand_rot)
        data=tfs.ToTensor()(data)
        data=tfs.Normalize(mean=[0.64, 0.6, 0.58],std=[0.14,0.15, 0.152])(data)
        target=tfs.ToTensor()(target)
        return  data ,target
    def __len__(self):
        return len(self.haze_imgs)

1
/content/drive/My Drive/code_custom_model
crop size whole_img
./data_fix_real/data_train/GT
crop size whole img
./data_fix_real/data_test/level_1/GT


In [ ]:
# define training and testing path
data_train_path = './data_fix_real/data_train'
data_test_path = './data_fix_real/data_test/level_1'

In [ ]:
# call data loader
OTS_train_loader=DataLoader(dataset=RESIDE_Dataset(data_train_path, train=True, format='.jpg'),batch_size=BS,shuffle=True)
OTS_test_loader=DataLoader(dataset=RESIDE_Dataset(data_test_path, train=False, size='whole img',format='.jpg'),batch_size=1,shuffle=False)


# Metrics

In [ ]:
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


def ssim(img1, img2, window_size=11, size_average=True):
    img1=torch.clamp(img1,min=0,max=1)
    img2=torch.clamp(img2,min=0,max=1)
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    return _ssim(img1, img2, window, window_size, channel, size_average)
def psnr(pred, gt):
    pred=pred.clamp(0,1).cpu().numpy()
    gt=gt.clamp(0,1).cpu().numpy()
    imdff = pred - gt
    rmse = math.sqrt(np.mean(imdff ** 2))
    if rmse == 0:
        return 100
    return 20 * math.log10( 1.0 / rmse)

# Perceptual Loss

In [ ]:
# --- Perceptual loss network  --- #
class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3"
        }

    def output_features(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return list(output.values())

    def forward(self, dehaze, gt):
        loss = []
        dehaze_features = self.output_features(dehaze)
        gt_features = self.output_features(gt)
        for dehaze_feature, gt_feature in zip(dehaze_features, gt_features):
            loss.append(F.l1_loss(dehaze_feature, gt_feature))

        return sum(loss)/len(loss)

# Modelling

In [ ]:
def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size,padding=(kernel_size//2), bias=bias)
    

# --- Build dense --- #
class MakeDense(nn.Module):
    def __init__(self, in_channels, growth_rate, kernel_size=3):
        super(MakeDense, self).__init__()
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=kernel_size, padding=(kernel_size-1)//2)

    def forward(self, x):
        out = F.relu(self.conv(x))
        out = torch.cat((x, out), 1)
        return out

# --- Build the Residual Dense Block --- #
class RDB(nn.Module):
    def __init__(self, in_channels, num_dense_layer, growth_rate):
        """
        :param in_channels: input channel size
        :param num_dense_layer: the number of RDB layers
        :param growth_rate: growth_rate
        """
        super(RDB, self).__init__()
        _in_channels = in_channels
        modules = []
        for i in range(num_dense_layer):
            modules.append(MakeDense(_in_channels, growth_rate))
            _in_channels += growth_rate
        self.residual_dense_layers = nn.Sequential(*modules)
        self.conv_1x1 = nn.Conv2d(_in_channels, in_channels, kernel_size=1, padding=0)

    def forward(self, x):
        out = self.residual_dense_layers(x)
        out = self.conv_1x1(out)
        out = out + x
        return out

# Build PA Layer
class PALayer(nn.Module):
  def __init__(self, channel):
    super(PALayer, self).__init__()
    self.pa = nn.Sequential(
        nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
        nn.ReLU(inplace=True),
        nn.Conv2d(channel // 8, 1, 1, padding=0, bias=True),
        nn.Sigmoid()
        )
  def forward(self, x):
    y = self.pa(x)
    return x * y

# Build CA Layer
class CALayer(nn.Module):
  def __init__(self, channel):
    super(CALayer, self).__init__()
    self.avg_pool = nn.AdaptiveAvgPool2d(1)
    self.ca = nn.Sequential(
        nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
        nn.ReLU(inplace=True),
        nn.Conv2d(channel // 8, channel, 1, padding=0, bias=True),
        nn.Sigmoid()
        )
  def forward(self, x):
    y = self.avg_pool(x)
    y = self.ca(y)
    return x * y

# Build Preprocessing Module
class PreprocessingModule(nn.Module):
  def __init__(self, in_channel, out_channel, kernel_size):
    super(PreprocessingModule, self).__init__() 
    if kernel_size ==3:
      self.conv1 =  nn.Conv2d(in_channel, out_channel, kernel_size,padding=(kernel_size//2), bias=True)
    elif kernel_size == 5:
      self.conv1 =  nn.Conv2d(in_channel, out_channel, kernel_size,padding=(kernel_size//2), bias=True)
    else:
      self.conv1 =  nn.Conv2d(in_channel, out_channel, kernel_size,padding=(kernel_size//2), bias=True)
    self.rdb =  RDB(in_channels= out_channel, num_dense_layer=4, growth_rate=out_channel )
  
  def forward(self, x):
    x = self.conv1(x)
    out = self.rdb(x)
    return out

# Build post-processing module
class PostprocessingModule(nn.Module):
  def __init__(self, channel, kernel_size):
    super(PostprocessingModule, self).__init__() 
    self.conv1 =  nn.Conv2d(channel, 3, kernel_size,padding=(kernel_size//2), bias=True)
    self.rdb =  RDB(in_channels= channel, num_dense_layer=4, growth_rate=64)
  
  def forward(self, x):
    x = self.rdb(x)
    out = self.conv1(x)
    return out

class HazeRemovalSubNetwork(nn.Module):
  def __init__(self, channel, kernel_size, dilation=1):
    super(HazeRemovalSubNetwork, self).__init__()
    if dilation == 1:
      self.conv_64 = nn.Conv2d(in_channels = channel,
                           out_channels = channel, 
                           kernel_size=kernel_size,
                           padding=(kernel_size//2),
                           bias=True)
    else:
      self.conv_64 = nn.Conv2d(in_channels = channel,
                           out_channels = channel, 
                           kernel_size=kernel_size,
                           dilation = dilation,
                           padding=dilation,
                           bias=True)
    self.palayer = PALayer(channel = channel)
    self.calayer = CALayer(channel = channel)
  def forward(self, x):
    #core line
    core = F.relu(self.conv_64(x))
    core1 = F.relu(self.conv_64(core + x))
    core2 = F.relu(self.conv_64(core1 + core + x))
    core3 = F.relu(self.conv_64(core2 + core1 + core + x))
    sum1 = x + core + core1 + core2 + core3

    core4 = F.relu(self.conv_64(sum1))
    core5 = F.relu(self.conv_64(core4 + sum1))
    core6 = F.relu(self.conv_64(core5 + core4 + sum1))
    core7 = F.relu(self.conv_64(core6 + core5 + core4 + sum1))
    sum2 = sum1 + core4 + core5 + core6 + core7

    core8 = F.relu(self.conv_64(sum2))
    #skip connection line
    skip = F.relu(self.conv_64(x))
    core8 = self.calayer(core8)
    core8 = self.palayer(core8)
    sum3 = core8 + skip

    out = self.conv_64(sum3)
    return out

class RefinementSubNetwork(nn.Module):
  def __init__(self,  channel, kernel_size, dilation=1):
    super(RefinementSubNetwork, self).__init__()
    if dilation == 1:
      self.conv_32_32 = nn.Conv2d(in_channels = channel,
                           out_channels = channel, 
                           kernel_size=kernel_size,
                           padding=(kernel_size//2),
                           bias=True)
    else:
      self.conv_32_32 = nn.Conv2d(in_channels = channel,
                           out_channels = channel, 
                           kernel_size=kernel_size,
                           dilation = dilation,
                           padding=dilation,
                           bias=True)
    self.palayer = PALayer(channel = channel)
    self.calayer = CALayer(channel = channel)
  def forward(self, x):
    #core line
    core = F.relu(self.conv_32_32(x))
    core1 = F.relu(self.conv_32_32(core))
    core2 = F.relu(self.conv_32_32(core1))
    core3 = F.relu(self.conv_32_32(core2))
    sum1 = x + core + core1 + core2 + core3

    core4 = F.relu(self.conv_32_32(sum1))
    core5 = F.relu(self.conv_32_32(core4))
    core6 = F.relu(self.conv_32_32(core5))
    core7 = F.relu(self.conv_32_32(core6))
    sum2 = sum1 + core4 + core5 + core6 + core7

    core8 = F.relu(self.conv_32_32(sum2))
    #skip connection line
    skip = F.relu(self.conv_32_32(x))
    # core8 = self.calayer(core8)
    # core8 = self.palayer(core8)
    sum3 = core8 + skip

    out = self.conv_32_32(sum3)
    return out

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.preprocessing_haze_sub1 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=3)
    self.preprocessing_haze_sub2 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=5)
    self.preprocessing_haze_sub3 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=7)
    self.hazeremovalsubnetwork_sub1 = HazeRemovalSubNetwork(channel=64, kernel_size=3)
    self.hazeremovalsubnetwork_sub2 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 2)
    self.hazeremovalsubnetwork_sub3 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 4)
    self.hazeremovalsubnetwork_sub4 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 8)
    self.hazeremovalsubnetwork_sub5 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 16)
    self.hazeremovalsubnetwork_sub6 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 32)
    self.hazeremovalsubnetwork_sub7 = HazeRemovalSubNetwork(channel=64, kernel_size=3, dilation = 64)

    self.palayer_haze_sub1 = PALayer(channel = 64)
    self.calayer_haze_sub1 = CALayer(channel = 64)
    self.palayer_haze_sub2 = PALayer(channel = 64)
    self.calayer_haze_sub2 = CALayer(channel = 64)
    self.palayer_haze_sub3 = PALayer(channel = 64)
    self.calayer_haze_sub3 = CALayer(channel = 64)

    self.preprocessing_refine_sub1 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=3)
    self.preprocessing_refine_sub2 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=5)
    self.preprocessing_refine_sub3 = PreprocessingModule(in_channel=3, out_channel=64, kernel_size=7)
    self.palayer_refine_sub1 = PALayer(channel = 64)
    self.calayer_refine_sub1 = CALayer(channel = 64)
    self.palayer_refine_sub2 = PALayer(channel = 64)
    self.calayer_refine_sub2 = CALayer(channel = 64)
    self.palayer_refine_sub3 = PALayer(channel = 64)
    self.calayer_refine_sub3 = CALayer(channel = 64)
    self.refinementsubnetwork_sub1 = RefinementSubNetwork(channel=64, kernel_size=3)
    self.refinementsubnetwork_sub2 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=2)
    self.refinementsubnetwork_sub3 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=4)
    self.refinementsubnetwork_sub4 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=8)
    self.refinementsubnetwork_sub5 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=16)
    self.refinementsubnetwork_sub6 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=32)
    self.refinementsubnetwork_sub7 = RefinementSubNetwork(channel=64, kernel_size=3, dilation=64)

    self.postprocessing_refine_sub1 = PostprocessingModule(channel=64, kernel_size=3)
    self.postprocessing_refine_sub2 = PostprocessingModule(channel=64, kernel_size=5)
    self.postprocessing_refine_sub3 = PostprocessingModule(channel=64, kernel_size=7)

    self.postprocessing_last = PostprocessingModule(channel=3, kernel_size=3)
    self.postprocessing_haze_last = PostprocessingModule(channel=64, kernel_size=3)
    self.conv_64_64 = default_conv(64,64,3)
    self.conv_32_32 = default_conv(32,32,3)
    self.conv_16_16 = default_conv(16,16,3)
    self.conv_64_3 = default_conv(64,3,3)
    self.conv_32_3 = default_conv(32,3,3)
    self.conv_16_3 = default_conv(16,3,3)
    self.conv_3_3 = default_conv(3,3,3)
    self.conv_3_64 = default_conv(3,64,3)
    
  def forward(self, x):
    

    #hazesub1
    preprocess_haze_sub1 = self.preprocessing_haze_sub1(x)

    hazeremoval_sub1_4 = self.hazeremovalsubnetwork_sub1(preprocess_haze_sub1)
    calayer_haze_sub1_5 = self.calayer_haze_sub1(hazeremoval_sub1_4)
    palayer_haze_sub1_6 = self.palayer_haze_sub1(calayer_haze_sub1_5)
    
    hazeremoval_sub2_4 = self.hazeremovalsubnetwork_sub2(preprocess_haze_sub1)
    calayer_haze_sub2_5 = self.calayer_haze_sub1(hazeremoval_sub2_4)
    palayer_haze_sub2_6 = self.palayer_haze_sub1(calayer_haze_sub2_5)

    hazeremoval_sub3_4 = self.hazeremovalsubnetwork_sub3(preprocess_haze_sub1)
    calayer_haze_sub3_5 = self.calayer_haze_sub1(hazeremoval_sub3_4)
    palayer_haze_sub3_6 = self.palayer_haze_sub1(calayer_haze_sub3_5)

    hazeremoval_sub4_4 = self.hazeremovalsubnetwork_sub4(preprocess_haze_sub1)
    calayer_haze_sub4_5 = self.calayer_haze_sub1(hazeremoval_sub4_4)
    palayer_haze_sub4_6 = self.palayer_haze_sub1(calayer_haze_sub4_5)

    concat_haze_refine_1 = palayer_haze_sub1_6 + palayer_haze_sub2_6 + palayer_haze_sub3_6 + palayer_haze_sub4_6 
   
    calayer_haze_sub = self.calayer_haze_sub1(concat_haze_refine_1)
    palayer_haze_sub = self.palayer_haze_sub1(calayer_haze_sub)
    conv_haze_sub1 = self.conv_64_64(palayer_haze_sub)
    conv_haze_sub2 = self.conv_64_64(conv_haze_sub1)

    conv_pass_haze_sub1 = self.conv_3_64(x)


    concat1_haze_sub1 = conv_haze_sub2 + conv_pass_haze_sub1
  
    out = self.postprocessing_haze_last(concat1_haze_sub1)

    #refine
    preprocessing_refine_sub1 = self.preprocessing_refine_sub1(out)

    refine_sub5_4 = self.refinementsubnetwork_sub4(preprocessing_refine_sub1)
    calayer_refine_sub5_5 = self.calayer_refine_sub1(refine_sub5_4)
    palayer_refine_sub5_6 = self.palayer_refine_sub1(calayer_refine_sub5_5)

    refine_sub6_4 = self.refinementsubnetwork_sub2(preprocessing_refine_sub1)
    calayer_refine_sub6_5 = self.calayer_refine_sub1(refine_sub6_4)
    palayer_refine_sub6_6 = self.palayer_refine_sub1(calayer_refine_sub6_5)

    refine_sub7_4 = self.refinementsubnetwork_sub3(preprocessing_refine_sub1)
    calayer_refine_sub7_5 = self.calayer_refine_sub1(refine_sub7_4)
    palayer_refine_sub7_6 = self.palayer_refine_sub1(calayer_refine_sub7_5)
    
    concat_refine_sub_1 = palayer_refine_sub5_6 + palayer_refine_sub6_6 + palayer_refine_sub7_6

    calayer_refine_sub = self.calayer_refine_sub1(concat_refine_sub_1)
    palayer_refine_sub = self.palayer_refine_sub1(calayer_refine_sub)
    conv_refine_sub1 = self.conv_64_64(palayer_refine_sub)
    conv_refine_sub2 = self.conv_64_64(conv_refine_sub1)

    conv_pass_refine_sub1 = self.conv_3_64(out)
    concat1_haze_sub1 = conv_refine_sub2 + conv_pass_refine_sub1

    out = self.postprocessing_refine_sub1(concat1_haze_sub1)
    return out


# Visualize the Model

In [ ]:
net=Net()

# Training

In [ ]:

def lr_schedule_cosdecay(t,T,init_lr=opt.lr):
    lr=0.5*(1+math.cos(t*math.pi/T))*init_lr
    return lr

def train(net,loader_train,loader_test,optim,criterion):
    losses=[]
    best_model_saved_step =0
    start_step=0
    max_ssim=0
    max_psnr=0
    ssims=[]
    psnrs=[]
    if opt.resume and os.path.exists(opt.model_dir):
        #load best model
        print(f'resume from {opt.model_dir}')
        ckp=torch.load(opt.model_dir)
        losses=ckp['losses']
        net.load_state_dict(ckp['model'])
        start_step=ckp['step']
        best_model_saved_step = ckp['step']
        max_ssim=ckp['max_ssim']
        max_psnr=ckp['max_psnr']
        psnrs=ckp['psnrs']
        ssims=ckp['ssims']
        print(f'best last_ssim :', ckp['losses'])
        #load last step training
        ckp_last_step = torch.load (last_save_model_dir)
        if ckp_last_step['step'] > ckp['step']:
            print(f'starting from last step')
            losses=ckp_last_step['losses']
            net.load_state_dict(ckp_last_step['model'])
            start_step=ckp_last_step['step']
            # max_ssim=ckp_last_step['max_ssim']
            # max_psnr=ckp_last_step['max_psnr']
            psnrs=ckp_last_step['psnrs']
            ssims=ckp_last_step['ssims']
        else:
            print(f'Starting from best SSIM')
            
        print(f'start_step:{start_step} start training ---')

    else :
        print('train from scratch *** ')

    for step in range(start_step+1,opt.steps+1):
        net.train()
        lr=opt.lr
        if not opt.no_lr_sche:
            lr=lr_schedule_cosdecay(step,T)
            for param_group in optim.param_groups:
                param_group["lr"] = lr  
        x,y=next(iter(loader_train))
        x=x.to(opt.device);y=y.to(opt.device)
        out=net(x)
        loss=criterion[0](out,y)
        # loss=criterion[0](out,y)
        # if opt.perloss:
        loss2=criterion[1](out,y)
        loss=loss+ torch.multiply(torch.tensor(0.04),loss2) +torch.sub(torch.tensor(1),  ssim(out, y))
        loss.backward()
        optim.step()
        optim.zero_grad()
        losses.append(loss.item())
        print(f'\rtrain loss : {loss.item():.5f}| step :{step}/{opt.steps}|lr :{lr :.7f} |time_used :{(time.time()-start_time)/60 :.1f}',end='',flush=True)
        print()

        #with SummaryWriter(logdir=log_dir,comment=log_dir) as writer:
        #writer.add_scalar('data/loss',loss,step)

        if step % (57//opt.bs) ==0 :
            with torch.no_grad():
                ssim_eval,psnr_eval=test(net,loader_test, max_psnr,max_ssim,step)

            print(f'\nstep :{step} |ssim:{ssim_eval:.4f}| psnr:{psnr_eval:.4f}')

            ssims.append(ssim_eval)
            psnrs.append(psnr_eval)
            if ssim_eval > max_ssim and psnr_eval > max_psnr :
                max_ssim=max(max_ssim,ssim_eval)
                max_psnr=max(max_psnr,psnr_eval)
                best_model_saved_step = step
                torch.save({
                            'step':step,
                            'max_psnr':max_psnr,
                            'max_ssim':max_ssim,
                            'ssims':ssims,
                            'psnrs':psnrs,
                            'losses':losses,
                            'model':net.state_dict()
                },opt.model_dir)
                print(f'\n model saved to best at step :{step}| max_psnr:{max_psnr:.4f}|max_ssim:{max_ssim:.4f}')
            else :
                # max_ssim=max(max_ssim,ssim_eval)
                # max_psnr=max(max_psnr,psnr_eval)
                torch.save({
                            'step':step,
                            'max_psnr':psnr_eval,
                            'max_ssim':ssim_eval,
                            'ssims':ssims,
                            'psnrs':psnrs,
                            'losses':losses,
                            'model':net.state_dict()
                }, last_save_model_dir)
                print(f'\n model saved to checkpoint at step :{step}| psnr:{psnr_eval:.4f}|ssim:{ssim_eval:.4f}')
                print(f'\n Last model saved to best at step :{best_model_saved_step}|  max_psnr:{max_psnr:.4f}|max_ssim:{max_ssim:.4f}')
    np.save(f'./numpy_files/{model_name}_{opt.steps}_losses.npy',losses)
    np.save(f'./numpy_files/{model_name}_{opt.steps}_ssims.npy',ssims)
    np.save(f'./numpy_files/{model_name}_{opt.steps}_psnrs.npy',psnrs)

def test(net,loader_test,max_psnr,max_ssim,step):
    net.eval()
    torch.cuda.empty_cache()
    ssims=[]
    psnrs=[]
    #s=True
    for i ,(inputs,targets) in enumerate(loader_test):
        inputs=inputs.to(opt.device);targets=targets.to(opt.device)
        pred=net(inputs)
        # # print(pred)
        # tfs.ToPILImage()(torch.squeeze(targets.cpu())).save('111.png')
        # vutils.save_image(targets.cpu(),'target.png')
        # vutils.save_image(pred.cpu(),'pred.png')
        ssim1=ssim(pred,targets).item()
        psnr1=psnr(pred,targets)
        ssims.append(ssim1)
        psnrs.append(psnr1)
        #if (psnr1>max_psnr or ssim1 > max_ssim) and s :
        #ts=vutils.make_grid([torch.squeeze(inputs.cpu()),torch.squeeze(targets.cpu()),torch.squeeze(pred.clamp(0,1).cpu())])
        #vutils.save_image(ts,f'samples/{model_name}/{step}_{psnr1:.4}_{ssim1:.4}.png')
        #s=False
    return np.mean(ssims) ,np.mean(psnrs)

In [ ]:
print('log_dir :',log_dir)
print('model_name:',model_name)
models_={
    'mod_pdr_net':Net(),
}
loaders_={
    # 'its_train':ITS_train_loader,
    # 'its_test':ITS_test_loader,
    'ots_train':OTS_train_loader,
    'ots_test':OTS_test_loader
}
start_time=time.time()
T=opt.steps	

log_dir : logs/ots_train_custom_model_modif_data_424_3_16
model_name: ots_train_custom_model_modif_data_424_3_16


In [ ]:
loader_train=loaders_[opt.trainset]
loader_test=loaders_[opt.testset]
net=models_[opt.net]
net=net.to(opt.device)
if opt.device=='cuda':
  net=torch.nn.DataParallel(net)
  cudnn.benchmark=True
criterion = []
criterion.append(nn.L1Loss().to(opt.device))
vgg_model = vgg16(pretrained=True).features[:16]
vgg_model = vgg_model.to(opt.device)
for param in vgg_model.parameters():
    param.requires_grad = False
criterion.append(LossNetwork(vgg_model).to(opt.device))
optimizer = optim.Adam(params=filter(lambda x: x.requires_grad, net.parameters()),lr=opt.lr, betas = (0.9, 0.999), eps=1e-08)
optimizer.zero_grad()
train(net,loader_train,loader_test,optimizer,criterion)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


Streaming output truncated to the last 5000 lines.
train loss : 0.16671| step :37889/100000|lr :0.0001000 |time_used :251.6
train loss : 0.09772| step :37890/100000|lr :0.0001000 |time_used :251.6
train loss : 0.07650| step :37891/100000|lr :0.0001000 |time_used :251.6
train loss : 0.16711| step :37892/100000|lr :0.0001000 |time_used :251.6
train loss : 0.06825| step :37893/100000|lr :0.0001000 |time_used :251.6
train loss : 0.14818| step :37894/100000|lr :0.0001000 |time_used :251.7
train loss : 0.15799| step :37895/100000|lr :0.0001000 |time_used :251.7
train loss : 0.45178| step :37896/100000|lr :0.0001000 |time_used :251.7
train loss : 0.07212| step :37897/100000|lr :0.0001000 |time_used :251.7
train loss : 0.06866| step :37898/100000|lr :0.0001000 |time_used :251.7
train loss : 0.51596| step :37899/100000|lr :0.0001000 |time_used :251.7
train loss : 0.19207| step :37900/100000|lr :0.0001000 |time_used :251.8
train loss : 0.42576| step :37901/100000|lr :0.0001000 |time_used :251.8
